In [1]:
from data_preparation import *
from get

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Conv2D, Dropout, Flatten, MaxPooling1D, MaxPooling2D
from tensorflow.keras import regularizers

%load_ext autoreload
%autoreload 2

2023-09-04 13:56:06.177527: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-04 13:56:06.180046: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 13:56:06.219676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 13:56:06.221513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 13:56:06.808712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
folder_list = extract_file_list()

# Model

In [3]:
def initialize_model_CNN(input_shape):
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=input_shape))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))),
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=3, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [4]:
patterns_to_test = ['rising_wedge', 'falling_wedge', 'double_top', 'double_bottom']
results_dict = {}

# Rising Wedge

In [5]:
X_real, y_real = real_data_pattern(patterns_to_test[0], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[0], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[0], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [6]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

2023-09-04 13:56:32.376267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 13:56:32.376575: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
21/21 [==============================] - 1s 22ms/step - loss: 78815.4609 - mae: 82.3209 - val_loss: 11192.5234 - val_mae: 59.8950
Epoch 2/50
21/21 [==============================] - 0s 17ms/step - loss: 47457.8594 - mae: 59.9356 - val_loss: 7271.9541 - val_mae: 49.4516
Epoch 3/50
21/21 [==============================] - 0s 17ms/step - loss: 6116.8750 - mae: 41.9964 - val_loss: 4738.4331 - val_mae: 38.6527
Epoch 4/50
21/21 [==============================] - 0s 17ms/step - loss: 4263.0039 - mae: 33.0066 - val_loss: 3854.1729 - val_mae: 33.2860
Epoch 5/50
21/21 [==============================] - 0s 16ms/step - loss: 3433.5208 - mae: 28.8006 - val_loss: 2893.0708 - val_mae: 27.6338
Epoch 6/50
21/21 [==============================] - 0s 16ms/step - loss: 4662.8804 - mae: 28.1357 - val_loss: 2920.9451 - val_mae: 30.0621
Epoch 7/50
21/21 [==============================] - 0s 17ms/step - loss: 3368.1768 - mae: 27.6031 - val_loss: 2436.4617 - val_mae: 24.4511
Epoch 8/50
21/21 [======

In [8]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Rising Wedge'] = res[1]

11/11 [==============================] - 0s 7ms/step - loss: 1695.2888 - mae: 19.1589


# Falling wedge

In [9]:
X_real, y_real = real_data_pattern(patterns_to_test[1], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[1], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[1], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [10]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [11]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
14/14 [==============================] - 1s 21ms/step - loss: 45890.1172 - mae: 59.1812 - val_loss: 35349.7812 - val_mae: 54.9286
Epoch 2/50
14/14 [==============================] - 0s 14ms/step - loss: 20202.8594 - mae: 43.1435 - val_loss: 3069.6802 - val_mae: 34.8334
Epoch 3/50
14/14 [==============================] - 0s 14ms/step - loss: 2729.9827 - mae: 27.3508 - val_loss: 1726.2605 - val_mae: 23.0518
Epoch 4/50
14/14 [==============================] - 0s 14ms/step - loss: 2322.0635 - mae: 24.8524 - val_loss: 1487.7130 - val_mae: 20.1690
Epoch 5/50
14/14 [==============================] - 0s 14ms/step - loss: 2003.3007 - mae: 22.9183 - val_loss: 1662.6820 - val_mae: 21.2940
Epoch 6/50
14/14 [==============================] - 0s 14ms/step - loss: 1782.3853 - mae: 20.7513 - val_loss: 1134.5894 - val_mae: 19.4092
Epoch 7/50
14/14 [==============================] - 0s 13ms/step - loss: 1615.6536 - mae: 19.5727 - val_loss: 1549.7601 - val_mae: 28.1721
Epoch 8/50
14/14 [======

In [12]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Falling Wedge'] = res[1]

8/8 [==============================] - 0s 6ms/step - loss: 566.5504 - mae: 8.3757


# Double Top

In [13]:
X_real, y_real = real_data_pattern(patterns_to_test[2], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[2], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[2], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [14]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [15]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
14/14 [==============================] - 1s 20ms/step - loss: 125433.9844 - mae: 83.9072 - val_loss: 87086.0000 - val_mae: 82.6465
Epoch 2/50
14/14 [==============================] - 0s 14ms/step - loss: 124479.6641 - mae: 66.7362 - val_loss: 9852.6963 - val_mae: 51.6983
Epoch 3/50
14/14 [==============================] - 0s 14ms/step - loss: 127314.3203 - mae: 67.2082 - val_loss: 10760.0830 - val_mae: 54.9101
Epoch 4/50
14/14 [==============================] - 0s 14ms/step - loss: 81718.7656 - mae: 58.7368 - val_loss: 9976.2188 - val_mae: 51.2522
Epoch 5/50
14/14 [==============================] - 0s 13ms/step - loss: 9451.1807 - mae: 48.8008 - val_loss: 9087.3232 - val_mae: 50.5439
Epoch 6/50
14/14 [==============================] - 0s 13ms/step - loss: 8196.9521 - mae: 47.2277 - val_loss: 8238.4336 - val_mae: 47.2688
Epoch 7/50
14/14 [==============================] - 0s 13ms/step - loss: 7975.1040 - mae: 46.8488 - val_loss: 7410.0005 - val_mae: 43.7493
Epoch 8/50
14/14 [

In [16]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Double Top'] = res[1]

8/8 [==============================] - 0s 5ms/step - loss: 2303.3765 - mae: 21.6818


# Double Bottom

In [17]:
X_real, y_real = real_data_pattern(patterns_to_test[3], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[3], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[3], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [18]:
y_all = np.array(y_joined)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [19]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
13/13 [==============================] - 1s 22ms/step - loss: 37710.6484 - mae: 65.0741 - val_loss: 24384.3574 - val_mae: 57.3781
Epoch 2/50
13/13 [==============================] - 0s 13ms/step - loss: 60295.1641 - mae: 58.1094 - val_loss: 33461.7734 - val_mae: 50.6036
Epoch 3/50
13/13 [==============================] - 0s 14ms/step - loss: 7088.0059 - mae: 41.5439 - val_loss: 48618.6953 - val_mae: 51.0653
Epoch 4/50
13/13 [==============================] - 0s 13ms/step - loss: 6202.2456 - mae: 37.3218 - val_loss: 99242.1406 - val_mae: 59.1796
Epoch 5/50
13/13 [==============================] - 0s 13ms/step - loss: 5906.5986 - mae: 35.4861 - val_loss: 71573.9922 - val_mae: 53.6542
Epoch 6/50
13/13 [==============================] - 0s 13ms/step - loss: 4988.3330 - mae: 32.5109 - val_loss: 101380.5391 - val_mae: 55.3580


In [20]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

results_dict['Double Bottom'] = res[1]

7/7 [==============================] - 0s 5ms/step - loss: 13363.3760 - mae: 56.0527


# Results

In [21]:
results_df = pd.DataFrame(results_dict, index=[1])
results_df

,Rising Wedge,Falling Wedge,Double Top,Double Bottom
1,19.158934,8.375728,21.681757,56.05267


# Pattern Present or not?

In [22]:
def initialize_model_CNN(input_shape):
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=input_shape))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))),
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Rising Wedge

In [23]:
accuracy_dict = {}

In [24]:
X_real, y_real = real_data_pattern(patterns_to_test[0], model_type='categorise')

X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[0], model_type='categorise')

X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[0], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [25]:
y_all = np.array(y_joined)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [26]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
21/21 [==============================] - 1s 18ms/step - loss: 17.5871 - accuracy: 0.6487 - val_loss: 41.3097 - val_accuracy: 0.5276
Epoch 2/50
21/21 [==============================] - 0s 13ms/step - loss: 11.9635 - accuracy: 0.6780 - val_loss: 2.5220 - val_accuracy: 0.7239
Epoch 3/50
21/21 [==============================] - 0s 13ms/step - loss: 6.0466 - accuracy: 0.7550 - val_loss: 4.9449 - val_accuracy: 0.6994
Epoch 4/50
21/21 [==============================] - 0s 13ms/step - loss: 4.3268 - accuracy: 0.7411 - val_loss: 15.9230 - val_accuracy: 0.7485
Epoch 5/50
21/21 [==============================] - 0s 13ms/step - loss: 6.1632 - accuracy: 0.7812 - val_loss: 10.0824 - val_accuracy: 0.7546
Epoch 6/50
21/21 [==============================] - 0s 13ms/step - loss: 4.2986 - accuracy: 0.8336 - val_loss: 4.2438 - val_accuracy: 0.8344
Epoch 7/50
21/21 [==============================] - 0s 13ms/step - loss: 3.8454 - accuracy: 0.8274 - val_loss: 9.5308 - val_accuracy: 0.7730


In [27]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

accuracy_dict['Rising Wedge_class'] = res[1]

11/11 [==============================] - 0s 5ms/step - loss: 2.0989 - accuracy: 0.7443


# Falling Wedge

In [28]:
X_real, y_real = real_data_pattern(patterns_to_test[1], model_type='categorise')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[1], model_type='categorise')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[1], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [29]:
y_all = np.array(y_joined)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [30]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
14/14 [==============================] - 1s 21ms/step - loss: 8.1340 - accuracy: 0.5923 - val_loss: 5.7390 - val_accuracy: 0.7273
Epoch 2/50
14/14 [==============================] - 0s 13ms/step - loss: 7.0160 - accuracy: 0.8200 - val_loss: 5.9922 - val_accuracy: 0.9000
Epoch 3/50
14/14 [==============================] - 0s 14ms/step - loss: 0.7889 - accuracy: 0.9453 - val_loss: 0.1515 - val_accuracy: 0.9545
Epoch 4/50
14/14 [==============================] - 0s 15ms/step - loss: 0.5350 - accuracy: 0.8998 - val_loss: 4.5970 - val_accuracy: 0.8636
Epoch 5/50
14/14 [==============================] - 0s 15ms/step - loss: 4.9127 - accuracy: 0.9544 - val_loss: 2.4572 - val_accuracy: 0.9364
Epoch 6/50
14/14 [==============================] - 0s 14ms/step - loss: 0.9772 - accuracy: 0.9544 - val_loss: 3.3693 - val_accuracy: 0.9273
Epoch 7/50
14/14 [==============================] - 0s 14ms/step - loss: 1.0924 - accuracy: 0.9613 - val_loss: 0.1399 - val_accuracy: 0.9818
Epoch 8/50
14

In [31]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

accuracy_dict['Falling Wedge class'] = res[1]

8/8 [==============================] - 0s 5ms/step - loss: 1.7721 - accuracy: 0.9661


# Double Top

In [32]:
X_real, y_real = real_data_pattern(patterns_to_test[2], model_type='categorise')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[2], model_type='categorise')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[2], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [33]:
y_all = np.array(y_joined)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [34]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
14/14 [==============================] - 1s 21ms/step - loss: 3.6954 - accuracy: 0.5711 - val_loss: 1.4021 - val_accuracy: 0.4505
Epoch 2/50
14/14 [==============================] - 0s 14ms/step - loss: 7.4918 - accuracy: 0.6749 - val_loss: 0.7955 - val_accuracy: 0.6126
Epoch 3/50
14/14 [==============================] - 0s 14ms/step - loss: 0.9113 - accuracy: 0.7359 - val_loss: 0.5305 - val_accuracy: 0.7748
Epoch 4/50
14/14 [==============================] - 0s 13ms/step - loss: 0.4928 - accuracy: 0.8352 - val_loss: 0.3868 - val_accuracy: 0.9189
Epoch 5/50
14/14 [==============================] - 0s 13ms/step - loss: 0.3335 - accuracy: 0.9052 - val_loss: 0.2487 - val_accuracy: 0.9009
Epoch 6/50
14/14 [==============================] - 0s 13ms/step - loss: 0.3074 - accuracy: 0.8894 - val_loss: 0.3056 - val_accuracy: 0.8919
Epoch 7/50
14/14 [==============================] - 0s 13ms/step - loss: 0.2732 - accuracy: 0.9187 - val_loss: 0.2449 - val_accuracy: 0.9279
Epoch 8/50
14

In [35]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

accuracy_dict['Double Top class'] = res[1]

8/8 [==============================] - 0s 5ms/step - loss: 0.4617 - accuracy: 0.8992


# Double Bottom

In [36]:
X_real, y_real = real_data_pattern(patterns_to_test[3], model_type='categorise')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[3], model_type='categorise')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[3], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [37]:
y_all = np.array(y_joined)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_all, test_size=0.30)

X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

In [38]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

input_shape = X_train.shape[1:]
model = initialize_model_CNN(input_shape)

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
13/13 [==============================] - 1s 22ms/step - loss: 7.0896 - accuracy: 0.6474 - val_loss: 0.9206 - val_accuracy: 0.7400
Epoch 2/50
13/13 [==============================] - 0s 14ms/step - loss: 0.6944 - accuracy: 0.8060 - val_loss: 0.5355 - val_accuracy: 0.7200
Epoch 3/50
13/13 [==============================] - 0s 14ms/step - loss: 0.4546 - accuracy: 0.7960 - val_loss: 0.2661 - val_accuracy: 0.9200
Epoch 4/50
13/13 [==============================] - 0s 14ms/step - loss: 0.3188 - accuracy: 0.8892 - val_loss: 0.2460 - val_accuracy: 0.9000
Epoch 5/50
13/13 [==============================] - 0s 14ms/step - loss: 0.2917 - accuracy: 0.8967 - val_loss: 0.2290 - val_accuracy: 0.9200
Epoch 6/50
13/13 [==============================] - 0s 14ms/step - loss: 0.3272 - accuracy: 0.9219 - val_loss: 0.2881 - val_accuracy: 0.9100
Epoch 7/50
13/13 [==============================] - 0s 13ms/step - loss: 0.3013 - accuracy: 0.9093 - val_loss: 0.2542 - val_accuracy: 0.9000
Epoch 8/50
13

In [39]:
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.int16)
res = model.evaluate(X_test, y_test)

accuracy_dict['Double Bottom class'] = res[1]

7/7 [==============================] - 0s 5ms/step - loss: 0.3599 - accuracy: 0.8785


# Results #2

In [43]:
accuracy_dict = pd.DataFrame(accuracy_dict, index=[0])
display("Accuracy DF")
display(accuracy_dict)
display("MAE DF")
display(results_df)

'Accuracy DF'

,Rising Wedge_class,Falling Wedge class,Double Top class,Double Bottom class
0,0.744253,0.966102,0.89916,0.878505


'MAE DF'

,Rising Wedge,Falling Wedge,Double Top,Double Bottom
1,19.158934,8.375728,21.681757,56.05267


# Testing with augmented data